Helper functions

In [7]:
def show_each_action_occurance(dataset):
    sums = np.zeros(lenA)
    for sample in dataset:
        a = A.index(sample[1][0])
        sums[a]+=1
    print(f"Action occurances: {sums}")

def save_with_config(dataset, name):
    if   'D0_1' in name:
        CM = CM0_1
    elif 'D1_1' in name:
        CM = CM1_1
    elif 'D2_1' in name:
        CM = CM2_1
    elif 'D3_1' in name:
        CM = CM3_1
    elif 'D4_1' in name:
        CM = CM4_1
    elif 'D0' in name:
        CM = CM0
    elif 'D1' in name:
        CM = CM1
    elif 'D2' in name:
        CM = CM2
    elif 'D3' in name:
        CM = CM3
    else: raise Exception("not the right name of dataset")
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM,
    }
    np.save(f"{os.path.expanduser('~')}/Documents/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))

# Datasets: D0, D1, D2, D3

- Direct mapping + scene + target object
- 11 Actions + 9 Gestures
- 10000 draws
- CM is:
    - `D1` 2D `[11,9]`
    - `D2` 3D `[3,11,9]`
    - `D3` 4D `[2,3,11,9]`
    - `D0` 2D `[11,9]` diagonal matrix
- types, scene and other features are not used
- scene: n objects - initialized randomly
- target object generation type: **random**

In [8]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a
import os

In [9]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point'] #, 'two', 'three']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [5]:
CM0 = np.zeros((lenA,lenG)) # actions x gestures
                    # up,left,dwn,rght,fiv,grab,thum,rota,poin,two,thre
CM0[:,:] = np.array([[ 1,  .0, .0,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_up
                     [.0,   1, .0,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_left
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0, .0, .0], # move_down
                     [.0,  .0, .0,   1, .0, .0,   .0,  .0,  .0, .0, .0], # move_right
                     [.0,  .0, .0,  .0,  1, .0,   .0,  .0,  .0, .0, .0], # put
                     [.0,  .0, .0,  .0, .0,  1,   .0,  .0,  .0, .0, .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0, .0, .0], # pour
                     [.0,  .0, .0,  .0, .0, .0,   .0,   1,  .0, .0, .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,   1, .0, .0], # place
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0,  1, .0], # open
                     [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0, .0,  1]])# close

In [18]:
CM1_1 = np.zeros((lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                     # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1_1[:,:] =np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                      [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                      [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                      [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                      [.0,  .0, .0,  .0,  1, .0,   .0,  .0,  .0], # put
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                      [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                      [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                      [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,   1], # open
                      [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [11]:
CM2_1 = np.zeros((3, lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[0,:,:] =np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[1,:,:] =np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                       [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                       [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2_1[2,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [12]:
CM3_1 = np.zeros((2,3, lenA,lenG)) # users x object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,0,:,:] =np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,1,:,:] =np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                       [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                       [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[0,2,:,:] = np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 0: cup
                       # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,0,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,1,:,:]=np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                       [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                       [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3_1[1,2,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                       [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                       [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                       [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                       [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# closeD1_1

In [ ]:
CM4_1 = np.zeros((2,2,3, lenA,lenG)) # object_state x users x object types x intents x gestures
''' -------------- STATE 0 -------------- '''
# objtyp 0: cup
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,0,:,:] =  np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,1,:,:] =  np.array([[ 0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                               [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                               [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,2,:,:] =   np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 0: cup
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,0,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,1,:,:]=np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                           [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                           [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,2,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
''' -------------- STATE 1 -------------- '''
# objtyp 0: cup
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,0,:,:] =  np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,1,:,:] =  np.array([[ 0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0,  0,  .0,  1, .0,   .0,  .0,  .0], # put
                               [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                               [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                              # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 0,2,:,:] =   np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                               [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                               [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                               [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                               [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                               [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                               [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                               [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                               [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                               [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 0: cup
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,0,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,  .0], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,1,:,:]=np.array([[0,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  0,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,  .0, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                           [.0,  .0,  1,  .0, .0, .0,   .0,  .0,   0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0,  1, .0,    0,  .0,   0], # place
                           [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0]])# close
# objtyp 2: object = box
                          # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM4_1[0, 1,2,:,:]=np.array([[1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                           [.0,   0, .0,   1, .0, .0,    0,   0,  .0], # move_left
                           [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                           [.0,   1, .0,   0,  0, .0,   .0,  .0,  .0], # move_right
                           [.0,  .0, .0,  .0,  0,  0,   .0,  .0,  .0], # put
                           [.0,  .0, .0,  .0,  1,  0,   .0,  .0,  .0], # put_on
                           [.0,  .0, .0,  .0, .0, .0,    0,   0,  .0], # pour
                           [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                           [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                           [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [13]:
draws = 10000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

Dataset generation:
- D0 - diagonal 2d matrix 11 x 11
- D1 - more gestures generated as one actions
- D2 - D1 + type of an object 
- D3 - D2 + user
- user perform two gestures
    - D0_2gs - similar as D0
    - D1_2gs - similar as D1
    - D2_2gs - similar as D2
    - D3_2gs - similar as D3
- static 3 objects on the scene
    - D1_3obj - similar as D1
    - D2_3obj - similar as D2
    - D3_3obj - similar as D3
-Versions with ones and zeros 
    - D0_1
    - D1_1
    - D2_1
    - D3_1
    - D4_1 - added state of the object

In [26]:
dataset_name = 'D3_1'
n_of_performed_gestures = 1

In [27]:
def create_observation(s,TaTo, dataset_name, n_of_performed_gestures=1):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = np.random.randint(2)
    object_state = np.random.randint(2)

    if   'D0_1' in dataset_name:
        gesture_probs_intent = CM0_1[target_action_id,:]
    elif 'D1_1' in dataset_name:
        gesture_probs_intent = CM1_1[target_action_id,:]
    elif 'D2_1' in dataset_name:
        gesture_probs_intent = CM2_1[target_object_type_id,target_action_id,:]
    elif 'D3_1' in dataset_name:
        gesture_probs_intent = CM3_1[user_id,target_object_type_id,target_action_id,:]
    elif 'D4_1' in dataset_name:
        gesture_probs_intent = CM4_1[object_state,user_id,target_object_type_id,target_action_id,:]
    elif 'D1' in dataset_name:
        gesture_probs_intent = CM1[target_action_id,:]
    elif 'D2' in dataset_name:
        gesture_probs_intent = CM2[target_object_type_id,target_action_id,:]
    elif 'D3' in dataset_name:
        gesture_probs_intent = CM3[user_id,target_object_type_id,target_action_id,:]
    elif 'D0' in dataset_name:
        gesture_probs_intent = CM0[target_action_id,:]
    else: raise Exception("Wrong name")
    if sum(gesture_probs_intent) == 0: return False, False
    
    try:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=False)
    except ValueError:
        performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=n_of_performed_gestures,replace=True)

        
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [28]:
dataset = []
# Generate random action
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    if n_objects > 7: n_objects = 7
    if n_objects < 2: n_objects = 2
    init = ",".join(object_types_order_list[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s, TaTo, dataset_name, n_of_performed_gestures=n_of_performed_gestures)
    if gobs is False: continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    #if i % 1000 == 0: print(i)

In [29]:
save_with_config(dataset, name=dataset_name)

In [21]:
show_each_action_occurance(dataset)

Action occurances: [   0. 2087. 2823. 2015.  568.    0.   28. 1123. 1115.  241.    0.]


Some actions were not feasible and were not triggered as often, this can be a problem, when evaluating dataset.
- D0 action occurances: [   0. 1869. 2474. 1879.  489.  247.   23. 1032. 1077.  249.  661.]
- D1 action occurances: [   0. 1838. 2564. 1848.  473.  276.   32. 1061. 1040.  214.  644.]
- D2 action occurances: [   0. 1854. 2504. 1913.  507.  253.   23. 1047. 1009.  221.  653.]
- D3 action occurances: [   0. 1850. 2566. 1854.  489.  257.   31. 1029. 1010.  248.  666.]

D1_1 [   0. 2087. 2823. 2015.  568.    0.   28. 1123. 1115.  241.    0.]

In [22]:
s = Scene(init='from_dict', import_data=dataset[0][0])
obs = s.scene_to_observation(type=1)        

In [23]:
obs

array([1, 3, 3, 1, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0,
       3, 3, 0, 0, 0, 0, 1, 1, 0, 0])

In [3]:
from submodules.Scenes import Scene

In [51]:
s = Scene(init='drawer,cup')
s.u.selected_id

0

### The old CMs

In [ ]:
CM1 = np.zeros((lenA,lenG)) # actions x gestures
                    # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1[:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                     [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                     [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                     [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                     [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                     [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                     [.5,  .0, .0,  .0, .0, .5,   .0,  .0,  .0], # open
                     [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close

In [ ]:
CM2 = np.zeros((3, lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                       [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [ ]:
CM3 = np.zeros((2,3, lenA,lenG)) # users x object types x actions x gestures
CM3[0,0,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                         [.0,  .0,  0,  .0, .0, .0,    1,  .0,   0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,   0], # place
                         [ 0,  .0, .0,  .0, .0,  0,    0,   1,  .0], # open
                         [.0,  .0,  0,  .0,  1,  0,   .0,  .0,  .0]])# close
# user 0, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  1, .0,   .0,  .0,   0], # put
                         [.0,  .0,  0,  .0, .0, .0,   .0,  .0,   1], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,   0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                         [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# user 1, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[1,2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close